# Prédisez la consommation d'énergie des bâtiments dans la ville de Seattle.

<div align="left">
  <img src="https://user.oc-static.com/upload/2024/09/11/17260684381511_Capture%20d%E2%80%99e%CC%81cran%202024-09-11%20a%CC%80%2017.22.25.png" width="200px">
</div>

## **Objetif**: Prédire les **émissions de CO2** et la **consommation totale d’énergie** de **bâtiments non destinés à l’habitation**

- Auteur......: **Rafael CEREZO MARTIN**
- Date........: **Décembre 2025**

# <span style="color:red"> 🏆 Modélisation complète via sklearn.pipeline intégrant le Feature Engineering.</span>

# ⬇️ IMPORTATION DES LIBRAIRIES (STACK DE SCIENCE DES DONNÉES)

In [33]:
# ##############################################################################
# MODULES STANDARDS ET BIBLIOTHÈQUES TIERCES
# ##############################################################################

# Bibliothèques de base
import os                                     # Gestion du système de fichiers
import joblib                                 # Persistance des objets Python
import numpy             as np                # Calcul numérique optimisé
import pandas            as pd                # Manipulation de structures data

# Transformation et Preprocessing Sklearn
from   sklearn.pipeline  import Pipeline      # Enchaînement des transformations
from   sklearn.compose   import ColumnTransformer # Application par colonnes
from   sklearn.impute    import SimpleImputer # Gestion des valeurs manquantes
from   sklearn.preprocessing import (
    StandardScaler,                           # Normalisation des données
    OneHotEncoder,                            # Encodage catégoriel binaire
    FunctionTransformer                       # Transformations personnalisées
)

# Modèles et Évaluation
from   sklearn.dummy           import DummyRegressor        # Modèle de base
from   sklearn.ensemble        import RandomForestRegressor # Forêt aléatoire
from   sklearn.linear_model    import LinearRegression      # Régression linéaire
from   sklearn.svm             import SVR                   # Machine à vecteurs de support

from   sklearn.model_selection import (
    train_test_split,                         # Division du jeu de données
    cross_validate,                           # Validation croisée
    KFold                                     # Stratégie de découpage CV
)
from   sklearn.metrics   import (
    r2_score,                                 # Coefficient de détermination
    mean_squared_error                        # Erreur quadratique moyenne
)

# Encodages avancés
from   category_encoders import TargetEncoder # Encodage par la cible

# ✂️ Load and Split Dataset pour Train et Test

In [34]:
print("\n============================================================================")
print("ÉTAPE 4 : EXÉCUTION DU WORKFLOW DE MODÉLISATION")
print("============================================================================")
# Chargement et séparation initiale
df_raw               = pd.read_csv('2016_Building_Energy_Benchmarking.csv')
X_prep, y_prep       = preparation_initiale(df_raw)

X_train, X_test, y_train, y_test = train_test_split(
    X_prep, y_prep, test_size=0.2, random_state=42
)


ÉTAPE 4 : EXÉCUTION DU WORKFLOW DE MODÉLISATION
⚠️ Nettoyage Target...: 5 lignes supprimées


# ⬇️ 👤 METHODES POUR LE PRE-PIPELINE

In [35]:
# ##############################################################################
# ÉTAPE 0 : PRÉPARATION INITIALE (NETTOYAGE HORS-PIPELINE)
# ##############################################################################

def preparation_initiale(df, target='SiteEnergyUse(kBtu)'):
    """
    Réalise le nettoyage structurel et élimine les valeurs cibles manquantes.
    """
    df_work          = df.copy()

    # 0.1 Élimination critique : Lignes sans valeur cible (Target)
    # --------------------------------------------------------------------------
    nb_avant         = len(df_work)
    df_work          = df_work.dropna(subset=[target])
    nb_apres         = len(df_work)
    
    if nb_avant != nb_apres:
        print(f"⚠️ Nettoyage Target...: {nb_avant - nb_apres} lignes supprimées")

    # 0.2 Suppression des colonnes constantes ou non informatives
    # --------------------------------------------------------------------------
    cols_a_supprimer = ['DataYear', 'City', 'State', 'Comments', 
                        'Outlier', 'OSEBuildingID', 'Electricity(kWh)']
    
    df_work          = df_work.drop(columns=cols_a_supprimer, errors='ignore')

    # 0.3 Séparation des caractéristiques et de la cible
    # --------------------------------------------------------------------------
    y                = df_work[target]
    x                = df_work.drop(columns=[target])

    return x, y
    
# ##############################################################################
# ÉTAPE 1 : IDENTIFICATION ET TYPAGE DES COLONNES
# ##############################################################################

def identifier_colonnes(df_x):
    """
    Catégorise les variables pour orienter les stratégies de transformation.
    """
    # 1.1 Variables numériques continues (Consommation, Surface, Localisation)
    numeriques       = ['Latitude', 'Longitude', 'YearBuilt', 'NumberofBuildings',
                        'NumberofFloors', 'PropertyGFATotal', 
                        'PropertyGFAParking', 'PropertyGFABuilding(s)',
                        'LargestPropertyUseTypeGFA', 
                        'SecondLargestPropertyUseTypeGFA',
                        'ThirdLargestPropertyUseTypeGFA', 'ENERGYSTARScore',
                        'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
                        'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
                        'SteamUse(kBtu)', 'Electricity(kBtu)',
                        'NaturalGas(therms)', 'NaturalGas(kBtu)',
                        'TotalGHGEmissions', 'GHGEmissionsIntensity']

    # 1.2 Variables nécessitant une transformation logarithmique
    cols_log         = ['PropertyGFATotal', 'PropertyGFABuilding(s)',
                        'LargestPropertyUseTypeGFA', 'SiteEUI(kBtu/sf)', 
                        'SourceEUI(kBtu/sf)', 'SteamUse(kBtu)', 
                        'Electricity(kBtu)', 'NaturalGas(kBtu)',
                        'TotalGHGEmissions']

    # 1.3 Variables catégorielles (Basse et Haute cardinalité)
    cat_ohe          = ['PrimaryPropertyType', 'BuildingType', 
                        'CouncilDistrictCode']
    cat_target       = ['Neighborhood']

    # 1.4 Variables nécessitant un indicateur de présence de vide
    cols_indic       = ['SecondLargestPropertyUseTypeGFA', 
                        'ThirdLargestPropertyUseTypeGFA', 'ENERGYSTARScore']

    # Filtrage par intersection pour garantir la présence dans le DataFrame
    return {
        'num_cont'   : [c for c in numeriques if c in df_x.columns],
        'num_log'    : [c for c in cols_log  if c in df_x.columns],
        'cat_ohe'    : [c for c in cat_ohe   if c in df_x.columns],
        'cat_target' : [c for c in cat_target if c in df_x.columns],
        'indic_nan'  : [c for c in cols_indic if c in df_x.columns]
    }

# ##############################################################################
# ÉTAPE 2 : LOGIQUE DES TRANSFORMATEURS PERSONNALISÉS
# ##############################################################################

def log_transform(df_x):
    """Applique log(1+x) avec gestion des valeurs négatives."""
    df_x             = df_x.copy()
    for col in df_x.columns:
        if df_x[col].min() < 0:
            offset   = abs(df_x[col].min()) + 1
            df_x[col] = np.log1p(df_x[col] + offset)
        else:
            df_x[col] = np.log1p(df_x[col])
    return df_x

# ------------------------------------------------------------------------------

def winsorize_transform(df_x, lower=0.01, upper=0.99):
    """Limite les valeurs extrêmes aux percentiles définis."""
    df_x             = df_x.copy()
    for col in df_x.columns:
        q_low        = df_x[col].quantile(lower)
        q_high       = df_x[col].quantile(upper)
        df_x[col]    = df_x[col].clip(lower=q_low, upper=q_high)
    return df_x

# On crée des fonctions simples qui appellent les originales avec les bons paramètres
# Cela remplace les lambdas qui bloquent le joblib.dump
def wrapper_winsorize(df_x):
    return winsorize_transform(df_x, lower=0.01, upper=0.99)



# ------------------------------------------------------------------------------

def create_features(df_x):
    """Génère de nouvelles variables métier (Feature Engineering)."""
    df_x             = df_x.copy()

    # Ratios d'efficacité énergétique et environnementale
    if 'TotalGHGEmissions' in df_x.columns and 'PropertyGFATotal' in df_x.columns:
        df_x['GHG_Density']    = df_x['TotalGHGEmissions'] / \
                                 (df_x['PropertyGFATotal'] + 1)

    if 'SecondLargestPropertyUseTypeGFA' in df_x.columns:
        df_x['Multi_Usage']    = (df_x['SecondLargestPropertyUseTypeGFA'] > 0)\
                                 .astype(int)

    if 'ENERGYSTARScore' in df_x.columns:
        df_x['Energy_Level']   = pd.cut(df_x['ENERGYSTARScore'], 
                                        bins=[0, 50, 75, 100],
                                        labels=['Low', 'Med', 'High'],
                                        include_lowest=True).astype(str)
    return df_x

# ------------------------------------------------------------------------------

def create_missing_indicators(df_x, cols_to_flag):
    """Crée des drapeaux binaires pour les valeurs manquantes importantes."""
    df_x             = df_x.copy()
    for col in cols_to_flag:
        if col in df_x.columns:
            df_x[f'{col}_Nan'] = df_x[col].isna().astype(int)
    return df_x

# On crée des fonctions simples qui appellent les originales avec les bons paramètres
# Cela remplace les lambdas qui bloquent le joblib.dump
def wrapper_missing_indicators(df_x, cols_to_flag):
    return create_missing_indicators(df_x, cols_to_flag)
    
# ##############################################################################
# ÉTAPE 3 : CONSTRUCTION DU PIPELINE DE PRÉTRAITEMENT
# ##############################################################################

def build_preprocessing_pipeline(df_x, v_y=None):
    """Assemble les briques de transformation dans un ColumnTransformer."""
    info             = identifier_colonnes(df_x)

    # 3.1 Sous-pipeline : Numérique avec Log
    pipe_num_log     = Pipeline(steps=[
        ('log',      FunctionTransformer(log_transform, validate=False)),
        ('winsor',   FunctionTransformer(winsorize_transform, validate=False,
                                         kw_args={'lower': 0.01, 
                                                  'upper': 0.99})),
        ('imputer',  SimpleImputer(strategy='median')),
        ('scaler',   StandardScaler())
    ])

    # 3.2 Sous-pipeline : Catégoriel One-Hot
    pipe_cat_ohe     = Pipeline(steps=[
        ('imputer',  SimpleImputer(strategy='constant', fill_value='NULL')),
        ('ohe',      OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # 3.3 Sous-pipeline : Target Encoding
    pipe_target      = Pipeline(steps=[
        ('imputer',  SimpleImputer(strategy='constant', fill_value='NULL')),
        ('target',   TargetEncoder(smoothing=10, min_samples_leaf=20))
    ])

    # Construction des transformateurs
    dispatch         = [
        ('n_log',    pipe_num_log, info['num_log']),
        ('c_ohe',    pipe_cat_ohe, info['cat_ohe'])
    ]

    if v_y is not None and info['cat_target']:
        dispatch.append(('c_target', pipe_target, info['cat_target']))

    preprocessor     = ColumnTransformer(transformers=dispatch, remainder='drop')

    # Pipeline global intégrant le feature engineering
    return Pipeline(steps=[
        ('nan_flags', FunctionTransformer(create_missing_indicators, 
                                                  validate=False,
                                                  kw_args={'cols_to_flag': 
                                                           info['indic_nan']})),
        ('feat_eng',  FunctionTransformer(create_features, validate=False)),
        ('preproc',   preprocessor)
    ])



# 1. ⚙️ Initialisation du PRE-PIPELINE

In [36]:
# Initialisation du pipeline de traitement
pre_pipeline         = build_preprocessing_pipeline(X_train, y_train)

# 2. ⚙️ CONFIGURATION DU BENCHMARK

In [37]:
# ============================================================================
# CONFIGURATION DU PROTOCOLE D'ÉVALUATION (VALIDATION CROISÉE)
# ============================================================================
# Définition de la stratégie de validation croisée (5-Fold)
# On utilise shuffle=True pour garantir l'indépendance des échantillons.
# La constante SEED assure la reproductibilité du découpage.
cv               = KFold(
                   n_splits     = 5, 
                   shuffle      = True, 
                   random_state = 42
)

# Définition des métriques de performance pour la comparaison
# 'neg_' indique que Scikit-Learn maximise l'opposé de l'erreur.
scoring          = {
    "r2"         : "r2",                                # Précision R²
    "mae"        : "neg_mean_absolute_error",           # Erreur MAE (Moyenne)
    "rmse"       : "neg_root_mean_squared_error",       # Erreur RMSE (Écart)
}

# Définition des algorithmes à challenger
models               = {
    # Modèle de référence : prédit systématiquement la moyenne du jeu d'entraînement
    "Baseline (Moyenne)"  : DummyRegressor(strategy="mean"),

    # Modèle linéaire : cherche une relation directe entre les variables et la cible
    "Régression Linéaire" : LinearRegression(),

    # Modèle non-linéaire (SVR) : utilise des noyaux pour projeter les données dans un espace supérieur
    "SVR (RBF)"           : SVR(kernel="rbf", C=1.0, epsilon=0.1),

    # Modèle d'ensemble (Random Forest) : agrégation de 100 arbres de décision pour réduire la variance
    "Random Forest"       : RandomForestRegressor(
                            n_estimators = 100,             # Nombre d'arbres dans la forêt
                            max_depth    = 10,              # Limite de profondeur pour éviter l'overfit
                            random_state = 42,            # Graine pour la reproductibilité
                            n_jobs       = -1)              # Utilisation de tous les processeurs disponibles
}



# 3. ⏳ BENCHMARK DES MODÈLES : EXTRACTION DÉTAILLÉE ET ÉVALUATION

In [39]:
# ============================================================================
# BENCHMARK DES MODÈLES : EXTRACTION DÉTAILLÉE ET ÉVALUATION
# ============================================================================

# Initialisation de la structure de stockage
# Cette liste contiendra les dictionnaires de métriques pour chaque modèle.
results_list     = []                                        # Initialisation



print("\n============================================================================")
print("🤖 MODÉLISATION - Comparaison via Cross-Validation")
print("============================================================================")

for name, model in models.items():
    print(f"\n🔄 Cross-validation pour {name}...")
    
    # Création du pipeline complet : Prétraitement + Modèle
    # Cette structure évite toute fuite de données (Data Leakage).
    full_pipe        = Pipeline(steps=[("prep", pre_pipeline), ("model", model)])
    
    # Exécution de la validation croisée
    cv_res             = cross_validate(
                         full_pipe, 
                         X_train, 
                         y_train, 
                         cv                 = kf,
                         scoring            = scoring,
                         n_jobs             = -1,
                         return_train_score = True
    )
    
    # ------------------------------------------------------------------------
    # EXTRACTION EXPLICITE DES MÉTRIQUES (MOYENNES ET ÉCARTS-TYPES)
    # ------------------------------------------------------------------------
    
    # Métriques R² (Coefficient de détermination)
    r2_train_mean      = cv_res["train_r2"].mean()              # Moyenne Train
    r2_test_mean       = cv_res["test_r2"].mean()               # Moyenne CV
    r2_test_std        = cv_res["test_r2"].std()                # Écart-type CV
    
    # Métriques MAE (Erreur Absolue Moyenne)
    # Note : On multiplie par -1 car Scikit-Learn retourne des valeurs négatives
    mae_train_mean     = -cv_res["train_mae"].mean()            # Moyenne Train
    mae_test_mean      = -cv_res["test_mae"].mean()             # Moyenne CV
    mae_test_std       = cv_res["test_mae"].std()               # Écart-type CV
    
    # Métriques RMSE (Erreur Quadratique Moyenne Racine)
    rmse_train_mean    = -cv_res["train_rmse"].mean()           # Moyenne Train
    rmse_test_mean     = -cv_res["test_rmse"].mean()            # Moyenne CV
    rmse_test_std      = -cv_res["test_rmse"].std()             # Écart-type CV
    
    # Calcul de l'écart de généralisation (Overfit Gap)
    overfit_gap        = r2_train_mean - r2_test_mean           # Écart R²
    
    # ------------------------------------------------------------------------
    # AFFICHAGE DU RAPPORT DE PERFORMANCE
    # ------------------------------------------------------------------------
    print(f"✅ {name}")
    print(f"   R²   train..: {r2_train_mean:.4f} | CV: {r2_test_mean:.4f} "
          f"± {r2_test_std:.4f}")
    print(f"   MAE  train..: {mae_train_mean:.2f} | CV: {mae_test_mean:.2f} "
          f"± {mae_test_std:.2f}")
    print(f"   RMSE train..: {rmse_train_mean:.2f} | CV: {rmse_test_mean:.2f} "
          f"± {rmse_test_std:.2f}")
    
    # Diagnostic du comportement du modèle
    if   overfit_gap >  0.10:
        print(f"   ⚠️  Surapprentissage probable (écart R² = {overfit_gap:.4f})")
    elif overfit_gap < -0.05:
        print(f"   ⚠️  Sous-apprentissage possible")
    else:
        print(f"   ✅ Bon équilibre train/CV")
    
    # ------------------------------------------------------------------------
    # ARCHIVAGE DANS LA STRUCTURE DE DONNÉES FINALE
    # ------------------------------------------------------------------------
    results_list.append({
        "Modèle"          : name,
        "R² CV (mean)"    : r2_test_mean,
        "R² CV (std)"     : r2_test_std,
        "MAE CV (mean)"   : mae_test_mean,
        "MAE CV (std)"    : mae_test_std,
        "RMSE CV (mean)"  : rmse_test_mean,
        "RMSE CV (std)"   : rmse_test_std,
        "R² train (mean)" : r2_train_mean,
        "Overfit gap (R²)": overfit_gap
    })


🤖 MODÉLISATION - Comparaison via Cross-Validation

🔄 Cross-validation pour Baseline (Moyenne)...
✅ Baseline (Moyenne)
   R²   train..: 0.0000 | CV: -0.0010 ± 0.0010
   MAE  train..: 5405347.92 | CV: 5403874.72 ± 417144.62
   RMSE train..: 15271001.74 | CV: 14606281.39 ± -4614216.86
   ✅ Bon équilibre train/CV

🔄 Cross-validation pour Régression Linéaire...
✅ Régression Linéaire
   R²   train..: 0.4908 | CV: 0.4259 ± 0.1664
   MAE  train..: 3984754.81 | CV: 4490683.10 ± 189872.68
   RMSE train..: 10913064.24 | CV: 11215698.42 ± -4763989.50
   ✅ Bon équilibre train/CV

🔄 Cross-validation pour SVR (RBF)...
✅ SVR (RBF)
   R²   train..: -0.0477 | CV: -0.0629 ± 0.0263
   MAE  train..: 4121293.92 | CV: 4123643.17 ± 396142.28
   RMSE train..: 15628047.50 | CV: 14992698.65 ± -4554825.86
   ✅ Bon équilibre train/CV

🔄 Cross-validation pour Random Forest...
✅ Random Forest
   R²   train..: 0.9468 | CV: 0.6790 ± 0.2127
   MAE  train..: 349970.94 | CV: 914506.43 ± 262550.69
   RMSE train..: 353179

# 4. 👤 ANALYSE COMPARATIVE ET SÉLECTION DU MODÈLE OPTIMAL

In [40]:
# ============================================================================
# ANALYSE COMPARATIVE ET SÉLECTION DU MODÈLE OPTIMAL
# ============================================================================

# Création du DataFrame récapitulatif à partir des résultats collectés
# Le tri par R² CV (mean) place le modèle le plus performant au sommet.
comparison_df    = pd.DataFrame(results_list).sort_values(
                   by        = "R² CV (mean)", 
                   ascending = False
)

# Extraction automatique du vainqueur pour la suite du pipeline
best_model_name  = comparison_df.iloc[0]["Modèle"]
best_r2_score    = comparison_df.iloc[0]["R² CV (mean)"]

# ----------------------------------------------------------------------------
# AFFICHAGE DU CLASSEMENT FINAL
# ----------------------------------------------------------------------------
print("\n============================================================================")
print("📊 TABLEAU RÉCAPITULATIF (Performance Cross-Validation sur Train Set)")
print("============================================================================")

# Utilisation de to_string pour garantir l'alignement strict des colonnes
print(comparison_df.to_string(
      index      = False, 
      justify    = 'left', 
      float_format = lambda x: f"{x:.4f}" if isinstance(x, float) else x
))

print("============================================================================")
print(f"🏆 MEILLEUR MODÈLE DÉTECTÉ : {best_model_name}  (selon R² CV mean)")
print(f"📈 Score R² de référence ..: {best_r2_score:.4f}")
print("============================================================================")

# ############################################################################
# INTERPRÉTATION 
# ############################################################################
# Le choix du modèle ne doit pas se baser uniquement sur le R² moyen.
# Vérifiez également le 'R² CV (std)' : un modèle stable présente un 
# écart-type faible, garantissant sa robustesse sur des données inconnues.
# ############################################################################


📊 TABLEAU RÉCAPITULATIF (Performance Cross-Validation sur Train Set)
Modèle               R² CV (mean)  R² CV (std)  MAE CV (mean)  MAE CV (std)  RMSE CV (mean)  RMSE CV (std)  R² train (mean)  Overfit gap (R²)
      Random Forest  0.6790       0.2127        914506.4326   262550.6909    8597691.9446   -5266614.8115   0.9468          0.2678           
Régression Linéaire  0.4259       0.1664       4490683.1035   189872.6753   11215698.4195   -4763989.4994   0.4908          0.0649           
 Baseline (Moyenne) -0.0010       0.0010       5403874.7179   417144.6171   14606281.3938   -4614216.8551   0.0000          0.0010           
          SVR (RBF) -0.0629       0.0263       4123643.1744   396142.2756   14992698.6475   -4554825.8603  -0.0477          0.0152           
🏆 MEILLEUR MODÈLE DÉTECTÉ : Random Forest  (selon R² CV mean)
📈 Score R² de référence ..: 0.6790


# 5. ⏳ ENTRAÎNEMENT DU MODÈLE FINAL (PRODUCTION READY)

In [41]:
# ============================================================================
# ENTRAÎNEMENT DU MODÈLE FINAL (PRODUCTION READY)
# ============================================================================

print("\n============================================================================")
print(f"📦 ENTRAÎNEMENT DU MODÈLE OPTIMAL : {best_model_name}")
print("============================================================================")

# Extraction du préprocesseur complet (incluant Feature Engineering)
# On récupère l'objet pipeline de traitement que nous avons testé en CV
final_preprocessor = pre_pipeline

# Choix de l'estimateur (Meilleur modèle identifié à l'étape 6)
best_estimator     = models[best_model_name]

# Assemblage final
# Ici, 'final_preprocessor' contient déjà : nan_flags + feat_eng + preproc
best_pipeline      = Pipeline(steps=[
    ("preprocess", final_preprocessor),       # Tout le flux de traitement
    ("model",      best_estimator)            # L'intelligence prédictive
])

# Apprentissage (C'est ici que le modèle "apprend" de X_train)
# ------------------------------------------------------------------------------
print(f"\n🚀 Entraînement final de {best_model_name} sur le jeu complet...")
best_pipeline.fit(X_train, y_train)

# Mesure de performance sur les données jamais vues (X_test)
# ------------------------------------------------------------------------------
y_pred_final     = best_pipeline.predict(X_test)
best_r2_score    = r2_score(y_test, y_pred_final)

# Rapport de clôture du projet
# ------------------------------------------------------------------------------
print("\n============================================================================")
print("BILAN FINAL DU PIPELINE")
print("============================================================================")
print(f"  Modèle entraîné.....: {best_model_name}")
print(f"  Performance R² Test.: {best_r2_score:.4f}")
print("============================================================================")

# Sauvegarde du binaire (Prêt pour la production)
# ------------------------------------------------------------------------------
nom_fichier      = 'best_model_pipeline.pkl'

try:
    joblib.dump(best_pipeline, nom_fichier)
    print(f"\n✅ Pipeline sauvegardé : '{nom_fichier}'")
    print(f"   État du modèle.....: ENTRAÎNÉ (FIT OK)")
    print(f"   Taille disque......: {os.path.getsize(nom_fichier) / 1024:.2f} KB")
except Exception as e:
    print(f"\n❌ ÉCHEC DE LA SAUVEGARDE : {str(e)}")
# ############################################################################
# ANALYSE TECHNIQUE 
# ############################################################################
# Ce 'best_pipeline' est désormais un objet autonome. Il peut recevoir des 
# données brutes (raw data), les transformer selon les règles du 
# 'preprocessor' et fournir une prédiction sans intervention manuelle.
# ############################################################################


📦 ENTRAÎNEMENT DU MODÈLE OPTIMAL : Random Forest

🚀 Entraînement final de Random Forest sur le jeu complet...

BILAN FINAL DU PIPELINE
  Modèle entraîné.....: Random Forest
  Performance R² Test.: 0.2923

✅ Pipeline sauvegardé : 'best_model_pipeline.pkl'
   État du modèle.....: ENTRAÎNÉ (FIT OK)
   Taille disque......: 4119.89 KB
